In [3]:
%load_ext dotenv
%dotenv

import pandas as pd
import os
import json
import hashlib
import pgOperation
from sqlalchemy.types import VARCHAR, Float, Integer, Date, Numeric
pd.options.mode.chained_assignment = None

# All configuration from environment variables
pg = pgOperation.PgOperation(
    ip=os.getenv('PG_HOST'),
    port=os.getenv('PG_PORT'),
    user=os.getenv('PG_USER'),
    pwd=os.getenv('PG_PASSWORD'),
    db=os.getenv('PG_DATABASE'),
    schema=os.getenv('PG_SCHEMA', 'public')
)

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [4]:
# Demo: Read first 5 rows from game_results table
demo_df = pg.readSql('SELECT * FROM game_results LIMIT 5')
print(f"Successfully connected to database: {os.getenv('PG_DATABASE')}")
print(f"Table: game_results (first 5 rows)")
demo_df

Successfully connected to database: terraforming-mars
Table: game_results (first 5 rows)


/Users/oushuohuang/Documents/tfm-data/pgOperation.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,con=conn)


,game_id,seed_game_id,players,generations,game_options,scores,createtime
0,f7cbeb6cc687,None,4,8,None,"[{""corporation"":""Poseidon"",""playerScore"":69,""p...",2020-08-31 15:05:21
1,10d3f58364b6,None,2,12,None,"[{""corporation"":""Aridor"",""playerScore"":144,""pl...",2020-08-31 15:05:21
2,ff2c9f3f3e45,None,4,8,None,"[{""corporation"":""Inventrix (breakthrough)"",""pl...",2020-08-31 15:05:21
3,cf45c3f6faa2,None,2,11,None,"[{""corporation"":""Arklight (breakthrough)"",""pla...",2020-08-31 15:05:21
4,79cef4b6a297,None,2,8,None,"[{""corporation"":""Point Luna"",""playerScore"":60,...",2020-08-31 15:05:21


In [ ]:
game_result_df = pg.readTable('game_results')
game_result_df['createtime'] = pd.to_datetime(game_result_df['createtime'])

In [ ]:
ods_main_games = pg.readTable('ods_main_games')
ods_main_games['prop'] = ods_main_games['prop'].apply(lambda x:eval(x))
pd.json_normalize(ods_main_games['prop'])

In [ ]:
tmp_game_df = pd.melt(
    # pd.concat([pd.DataFrame(pd.json_normalize(x)) for x in game_df['prop']],ignore_index=True)
    pd.json_normalize(ods_main_games['prop'])
    .reset_index(), id_vars='index', value_name='player_data').dropna(subset=['player_data']).set_index('index', drop=True)
flat_game_df = (pd.json_normalize(tmp_game_df['player_data']).join(tmp_game_df.reset_index(drop=False)['index']).set_index('index')) \
    .join(ods_main_games.drop(['seed_game_id', 'game_options', 'prop'], axis=1)).reset_index()
flat_game_df['position'] = flat_game_df.sort_index() \
            .groupby(['game_id']) \
            .cumcount() + 1
flat_game_df['rank'] = flat_game_df.sort_values(['playerScore'], ascending=[False]) \
            .groupby(['game_id']) \
            .cumcount() + 1

    # remove firstletter in flat_game_df['player'] if start with ~ or @
flat_game_df['player'] = flat_game_df['player'].apply(lambda x: x[1:] if x[0] in ['~', '@', '～'] else x)
# flat_game_df player to be lowercase
flat_game_df['player'] = flat_game_df['player'].apply(lambda x: x.lower())

flat_game_df.drop('index', axis=1, inplace=True)
pg.writeDfToPg(flat_game_df, 'ods_game_results')
